In [1]:
import cv2
import numpy as np

# Detection Methods

In [2]:
def drawCenter(cx , cy , frame):
        cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)
    
def getMaxContourCenter(processedImage , frame):
    contours, _  = cv2.findContours(processedImage, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    if(len(contours)==0):
        return None
    max_contour = max(contours, key=cv2.contourArea)
    M = cv2.moments(max_contour)
    try:
        cx = int(M["m10"] / M["m00"])
        cy = int(M["m01"] / M["m00"])
        drawCenter(cx , cy , frame)
        for contour in contours:
            c = contour
            if(c is max_contour):
                cv2.drawContours(frame, c, -1, (0, 255 , 0 ), 3)
            else: 
                cv2.drawContours(frame, c, -1, (0, 0, 255), 3)
        return cx , cy
    except ZeroDivisionError:
        print("Zero Division Error")
        return None

# Global Config

In [3]:
kernel_size = 5
low_threshold = 10
high_threshold = 50

### Method of Canny

In [4]:
def canny(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    blur = cv2.GaussianBlur(gray, (kernel_size, kernel_size), 0)
    
    # Apply Canny edge detection
    # edges = cv2.Canny(blur, low_threshold, high_threshold)

    edges = cv2.Laplacian(blur, -1, ksize=5)

    edges = cv2.medianBlur(edges,9)

    # edges = cv2.GaussianBlur(edges, (kernel_size, kernel_size), 0)


    kernal = np.ones((3, 3), np.uint8)
    # edges = cv2.dilate(edges, kernal, iterations=10)
    edges = cv2.erode(edges, kernal, iterations=1)
    # edges = cv2.dilate(edges, kernal, iterations=10)

    cv2.imshow('canny', edges)

    return getMaxContourCenter(edges , frame)

### Method of houghLines

In [5]:
rho = 1  # distance resolution in pixels of the Hough grid
theta = np.pi / 180  # angular resolution in radians of the Hough grid
threshold = 15  # minimum number of votes (intersections in Hough grid cell)
min_line_length = 100  # minimum number of pixels making up a line
max_line_gap = 50  # maximum gap in pixels between connectable line segments

In [6]:
def houghLines(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur_gray = cv2.GaussianBlur(gray,(kernel_size, kernel_size),0)
    edges = cv2.Canny(blur_gray, low_threshold, high_threshold)
    

    # Run Hough on edge detected image
    # Output "lines" is an array containing endpoints of detected line segments
    lines = cv2.HoughLinesP(edges, rho, theta, threshold, np.array([]),
                        min_line_length, max_line_gap)


    line_image = np.copy(frame) * 0  # creating a blank to draw lines on
    if lines is None:
        return None
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(line_image,(x1,y1),(x2,y2),(255,0,0),5)
    
    cv2.imshow('hough', line_image)

### Using color Detection

In [7]:
#!TODO - specify the parameters
#!TODO - comment the windows for debugging

In [8]:
def detect_black(frame , width_tolerance = 300 , height_ratio = 0.5 , min_area = 20000):
    frame = cv2.medianBlur(frame, 5)

    black_only = cv2.inRange(frame, (0, 0, 0), (40, 40, 40))


    # Finding largest contour in the ROI
    contours , _ = cv2.findContours(black_only, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # print(contours)
    if(len(contours)==0):
    # Draw the shape on the image
        cv2.putText(frame, "false , false (no contours)" , (200,200), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        cv2.imshow("frame" , frame)
        return False , False
    max_contour = max(contours, key=cv2.contourArea)
    print(cv2.contourArea(max_contour))
    if(cv2.contourArea(max_contour) < min_area):
    # Draw the shape on the image
        cv2.putText(frame, "false , fasle (small contours)" , (200,200), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        cv2.imshow("frame" , frame)
        return False , False
    mask = np.zeros(frame.shape, np.uint8)
    cv2.drawContours(mask, [max_contour], -1, 255, -1)

    # Define ROI coordinates
    height, width = frame.shape[:2]
    black_line_left = mask[int(height*(1-height_ratio)):height, 0:int(width / 2)-int(width_tolerance/2)]  # Left side ROI
    black_line_right = mask[int(height*(1-height_ratio)):height, int(width / 2) + int(width_tolerance/2):width]  # Right side ROI

    # # Dilate to enhance detection
    # kernel = np.ones((3, 3), np.uint8)
    # black_line_left = cv2.dilate(black_line_left, kernel, iterations=2)
    # black_line_right = cv2.dilate(black_line_right, kernel, iterations=2)



    # Check for black lines presence
    left_contains_black = np.any(black_line_left)
    right_contains_black = np.any(black_line_right)

    # !NOTE - Put text is enough for debugging
    # # Visualization
    # combined_mask = np.hstack((black_line_left, black_line_right))  # Combine masks horizontally
    # cv2.imshow("Black Line Detection - Left and Right", combined_mask)
    # cv2.imshow("max-contour" , mask)

    # Draw the shape on the image
    cv2.putText(frame, f"{left_contains_black , right_contains_black}" , (200,200), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
    cv2.imshow("frame" , frame)
    return bool(left_contains_black), bool(right_contains_black)

### Using Thresholding

In [9]:
def thresh(frame):
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.medianBlur(gray,5)

    res = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,11,2)
    cv2.imshow('thresh', res)
    

In [10]:
cap = cv2.VideoCapture(0)
cap.open("http://192.168.1.3:8080/video")

if not cap.isOpened():
    print("Cannot open camera")
    exit()

while True:
    ret, frame = cap.read()
    # frame = cv2.rotate( frame, cv2.ROTATE_90_CLOCKWISE)
    frame = cv2.resize(frame, (640, 480))
    
    # canny(frame)
    # houghLines(frame)
    detect_black(frame)
    # thresh(frame)

    # cv2.imshow('frame', frame)
    key = cv2.waitKey(1) & 0xFF	
    if key == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

77579.0


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/abdo/.local/lib/python3.10/site-packages/cv2/qt/plugins"


(False, True)
76117.0
(False, True)
77823.5
(False, True)
77259.0
(False, True)
76693.5
(False, True)
74947.5
(False, True)
81316.0
(False, True)
76513.5
(False, True)
73280.0
(False, True)
75864.0
(False, True)
78889.5
(False, True)
78204.5
(False, True)
77901.0
(False, True)
80167.0
(False, True)
82130.5
(True, True)
72189.5
(True, True)
73576.0
(False, True)
77553.0
(False, True)
24831.0
(False, True)
17900.0
(False, False)
17187.0
(False, False)
12983.0
(False, False)
12213.5
(False, False)
10511.5
(False, False)
8680.0
(False, False)
6042.0
(False, False)
3909.0
(False, False)
2520.5
(False, False)
2520.5
(False, False)
2561.0
(False, False)
2438.0
(False, False)
2394.5
(False, False)
2296.5
(False, False)
2234.0
(False, False)
2205.0
(False, False)
1918.0
(False, False)
1833.5
(False, False)
2189.5
(False, False)
3109.5
(False, False)
4437.0
(False, False)
4103.0
(False, False)
4854.5
(False, False)
5773.5
(False, False)
6912.5
(False, False)
9248.5
(False, False)
11047.5
(False,

[mjpeg @ 0x5fecca651b00] overread 8


11864.5
(False, False)
12971.0
(False, False)
13486.5
(False, False)
21895.5
(False, False)
26127.5
(True, False)
40904.0
(True, True)
45284.5
(True, True)
39457.5
(True, True)
28933.0
(True, False)
30068.5
(True, False)
29081.0
(True, False)
746.0
(False, False)
17336.0
(False, False)
33372.0
(True, False)
53573.5
(True, True)
51874.5
(True, True)
52071.5
(True, True)
52347.5
(True, True)
40536.5
(True, False)
27830.5
(True, False)
27914.5
(False, False)
28349.0
(False, False)
11066.0
(False, False)
9349.0
(False, False)
6531.5
(False, False)
5527.5
(False, False)
3856.5
(False, False)
3075.0
(False, False)
2554.0
(False, False)
1502.5
(False, False)
1445.0
(False, False)
1385.0
(False, False)
1175.5
(False, False)
881.5
(False, False)
705.0
(False, False)
453.5
(False, False)
309.0
(False, False)
220.0
(False, False)
329.5
(False, False)
221.0
(False, False)
212.0
(False, False)
467.0
(False, False)
806.5
(False, False)
985.0
(False, False)
464.5
(False, False)
549.0
(False, False)
3

...

17514.0
(False, False)
17289.0
(False, False)
17575.5
(False, False)
18761.0
(False, False)
20458.0
(True, False)
20637.0
(True, False)
20536.0
(True, False)
18632.0
(False, False)
18243.5
(False, False)
17594.0
(False, False)
16712.0
(False, False)
16586.5
(False, False)
15766.0
(False, False)
15031.5
(False, False)
14851.0
(False, False)
15420.5
(False, False)
16994.5
(False, False)
24986.5
(True, False)
25870.0
(True, False)
29590.0
(True, False)
31227.5
(True, False)
30008.0
(True, False)
26982.5
(True, False)
24621.0
(True, False)
24773.0
(True, False)
25352.0
(True, False)
25091.0
(True, False)
24449.5
(True, False)
24845.0
(True, False)
24395.0
(True, False)
24244.5
(True, False)
24713.0
(True, False)
25035.0
(True, False)
24244.5
(True, False)
24306.0
(True, False)
24635.0
(True, False)
25630.5
(True, False)
26576.5
(True, False)
27508.5
(True, False)
28625.5
(True, False)
26569.5
(True, False)
4330.0
(False, False)
4083.5
(False, False)
1347.5
(False, False)
1974.5
(False, Fal

[mjpeg @ 0x5fecca651b00] overread 2


14908.0
(False, False)
14908.0
(False, False)
15548.5
(False, False)
21962.5
(False, False)
5881.0
(False, False)
6529.5
(False, False)
19537.5
(False, False)
31293.0
(False, False)
33480.5
(False, False)
23163.0
(False, False)
13165.0
(False, False)
4867.0
(False, False)
3455.0
(False, False)
2500.0
(False, False)
2989.0
(False, False)
3301.5
(False, False)
4662.5
(False, False)
5006.0
(False, False)
9073.5
(False, False)
6286.5
(False, False)
6380.5
(False, False)
6276.5
(False, False)
6278.0
(False, False)
6317.0
(False, False)
6105.5
(False, False)
5869.0
(False, False)
5399.0
(False, False)
4876.0
(False, False)
4923.0
(False, False)
4939.0
(False, False)
4431.0
(False, False)
3891.5
(False, False)
4100.5
(False, False)
4654.0
(False, False)
4836.0
(False, False)
4524.5
(False, False)
4440.5
(False, False)
4487.5
(False, False)
3514.0
(False, False)
2395.0
(False, False)
1994.0
(False, False)
3509.0
(False, False)
5135.0
(False, False)
9740.5
(False, False)
31137.0
(False, True)
3

[ WARN:0@187.535] global cap_ffmpeg_impl.hpp:453 _opencv_ffmpeg_interrupt_callback Stream timeout triggered after 30052.968105 ms


error: OpenCV(4.10.0) /io/opencv/modules/imgproc/src/resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'resize'
